In [5]:
from __future__ import annotations

from typing import Literal, Optional
from pydantic import BaseModel, Field
from dotenv import load_dotenv

from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# --- setup model ---
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# --- structured outputs ---

class SentimentState(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="sentiment of the review")

struct_sentiment = model.with_structured_output(SentimentState)

class Diagnosis(BaseModel):
    issue_title: str = Field(description="Short title for the issue being reported, if any")
    tone: str = Field(description="Writer's tone (e.g., angry, frustrated, appreciative, neutral)")
    urgency: bool = Field(description="True if it seems urgent to address, else False")

struct_diagnosis = model.with_structured_output(Diagnosis)

# --- graph state ---

class FlowState(BaseModel):
    review: str = Field(description="Review given by user")
    sentiment: Literal["positive", "negative"] | None = None
    issue_title: str | None = None
    tone: str | None = None
    urgency: bool | None = None
    response: Optional[str] = Field(default=None, description="Agent response to return")

# --- nodes ---

def find_sentiment(state: FlowState) -> dict:
    prompt = (
        "You will classify review sentiment as strictly 'positive' or 'negative'.\n"
        f"Review: {state.review!r}\n"
        "Output only the fields requested by the schema."
    )
    sentiment: SentimentState = struct_sentiment.invoke(prompt)
    return {"sentiment": sentiment.sentiment}

def run_diagnosis(state: FlowState) -> dict:
    prompt = (
        "Extract an issue title (if any), the author's tone, and whether it's urgent.\n"
        "If no clear issue is present, make a concise title like 'General Feedback'.\n"
        f"Review: {state.review!r}\n"
        "Decide 'urgency' as True only if this needs quick action (e.g., outage, payment failure, data loss)."
    )
    diag: Diagnosis = struct_diagnosis.invoke(prompt)
    return {
        "issue_title": diag.issue_title,
        "tone": diag.tone,
        "urgency": diag.urgency,
    }

def positive_response(state: FlowState) -> dict:
    reply = (
        "Thanks for the positive feedback! 🎉\n"
        "We're glad you had a good experience. If there's anything specific you loved or want more of, tell us!"
    )
    return {"response": reply}

def negative_response(state: FlowState) -> dict:
    # Craft a brief, empathetic reply using extracted fields if available
    parts = ["I'm sorry you had a rough experience."]
    if state.issue_title:
        parts.append(f"I've logged it as: {state.issue_title}.")
    if state.urgency is True:
        parts.append("We'll prioritize this and start looking into it right away.")
    else:
        parts.append("We'll investigate and follow up soon.")
    parts.append("Thanks for flagging this.")
    reply = " ".join(parts)
    return {"response": reply}

# --- routing ---

def check_sentiment(state: FlowState) -> str:
    # Return the next node key
    if state.sentiment == "positive":
        return "positive_response"
    return "run_diagnosis"

# --- build graph ---

graph = StateGraph(FlowState)

graph.add_node("find_sentiment", find_sentiment)
graph.add_node("run_diagnosis", run_diagnosis)
graph.add_node("negative_response", negative_response)
graph.add_node("positive_response", positive_response)

graph.add_edge(START, "find_sentiment")
graph.add_conditional_edges("find_sentiment", check_sentiment,  # maps to node names returned above
                            {"positive_response": "positive_response", "run_diagnosis": "run_diagnosis"})
graph.add_edge("run_diagnosis", "negative_response")
graph.add_edge("positive_response", END)
graph.add_edge("negative_response", END)

workflow= graph.compile()
workflow

# --- example run ---
if __name__ == "__main__":
    # Positive example
    state = FlowState(review="Loved the new update! Everything feels faster and cleaner.")
    out = workflow.invoke(state)
    print(out)

    # Negative example
    state2 = FlowState(review="Payment failed twice and I lost my discount. Super frustrating.")
    out2 = workflow.invoke(state2)
    print(out2)


{'review': 'Loved the new update! Everything feels faster and cleaner.', 'sentiment': 'positive', 'response': "Thanks for the positive feedback! 🎉\nWe're glad you had a good experience. If there's anything specific you loved or want more of, tell us!"}
{'review': 'Payment failed twice and I lost my discount. Super frustrating.', 'sentiment': 'negative', 'issue_title': 'Payment Failure and Lost Discount', 'tone': 'frustrated', 'urgency': True, 'response': "I'm sorry you had a rough experience. I've logged it as: Payment Failure and Lost Discount. We'll prioritize this and start looking into it right away. Thanks for flagging this."}


NameError: name 'flow_state' is not defined